In [11]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    var GoogleMapsView = widget.DOMWidgetView.extend({
        initialize: function() {
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                }
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'], 
                    function() {}, 
                    function() {});
            }
        },
        
        render: function() {
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $('<div style="height: 300px; width: 400px;"></div>');
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {lat: this.model.get('lat'), lng: this.model.get('lng')},
                        zoom: this.model.get('zoom')
                    });
                    
                    google.maps.event.addListener(this.map, 'bounds_changed', _.bind(this.syncFromMap, this));
                    
                    this.listenTo(this.model, 'change:lat', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:lng', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:zoom', _.bind(this.syncFromModel, this));
                }, this));
            }, this));
        },
        
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            this.model.save_changes();
            console.log(this.model);
        },
        
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        }
    });
    
    manager.WidgetManager.register_widget_view('GoogleMapsView', GoogleMapsView);
});

<IPython.core.display.Javascript object>

In [12]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget, traitlets.HasTraits):
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)

In [13]:
from IPython.display import display

a = GoogleMapsView()
b = GoogleMapsView()

widgets.jslink((a, 'lat'), (b, 'lat'))
widgets.jslink((a, 'lng'), (b, 'lng'))
widgets.jslink((a, 'zoom'), (b, 'zoom'))

In [16]:
a

In [17]:
b

In [20]:
a.lat

47.517200697839414

In [21]:
b.lat

47.517200697839414